In [ ]:
#import os to provide access to functionality dependent on Operating system, The sys module is tells about which Python script is interacting with the host system

import os, sys
from google.colab import drive
drive.mount('/content/drive')

#Define the path to download  the packages
nb_path = '/content/notebooks'

#Link the path where our notebooks is saved
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --target=$nb_path textsearch 

     |████████████████████████████████| 286kB 3.0MB/s 
     |████████████████████████████████| 327kB 8.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85387 sha256=11cd9d4451d87f38effc38def647ab8b2c758cf10c26dccc0e659b4133e7268d
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
!pip install --target=$nb_path contractions

  Using cached https://files.pythonhosted.org/packages/d3/fe/021d7d76961b5ceb9f8d022c4138461d83beff36c3938dc424586085e559/textsearch-0.0.21-py2.py3-none-any.whl
Processing /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842/pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a3/14/666cd44bf53f36a961544af592cb5c5c800013f9c51a4745af8d7c17362a/anyascii-0.2.0-py3-none-any.whl


In [ ]:
!pip install --target=$nb_path tqdm

     |████████████████████████████████| 81kB 2.7MB/s 


In [ ]:
!pip install --target=$nb_path num2words

     |████████████████████████████████| 102kB 2.9MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c0dc2a4acaec0bc24a2d82da030cfa28c968d52ad69ce90f77e3e7449045ee9b
  Stored in directory: /root/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


In [ ]:
# import nltk, csv, re packages
import nltk
import csv
import re

#Download the punkt package to remove punctuations
nltk.download('punkt')

#Download the stopwors package to remove common stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd                        #Import pandas package as pd
import contractions                        #Import contractions package
from num2words import num2words            #Import num2words package
from nltk.tokenize import word_tokenize    #Import the word_tokenizer package
from nltk.tokenize import regexp_tokenize  #Import the regexp_tokenizer package
from nltk.stem import WordNetLemmatizer    #Import the WordNetLemmatizer package
from nltk.stem import PorterStemmer        #Impport the Porter stemmer package

# Load the Chat messages with design parameters

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Engine_design/Designintent1.csv")

In [ ]:
df2 = pd.read_csv("/content/notebooks/Data/DesignIntent2.csv")

In [ ]:
df2.head()

,body,intent_2
0,Hi piston! My crankshaft depends on your bore ...,NAN
1,thank you!,NAN
2,How low can you go on the piston bore diameter?,Exploration of design parameter values
3,"all my fos values are in the hundreds, i suspe...",Effects of design parameters on objectives
4,Hi flywheel! My crankshaft depends on your fly...,Dependencies between design parameters


# Create the new dataframe with chat data

In [ ]:
df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df.drop("a",axis=1)

# Copy the chat conversations to new dataframe

In [ ]:
df1 = df[['body']].astype('str')

In [ ]:
df.drop_duplicates(keep=False, inplace=True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 629 entries, 0 to 738
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    629 non-null    object
dtypes: object(1)
memory usage: 9.8+ KB


#  Step1: Expanding the contracted words using contractions library

In [ ]:
df1['expan_contra']=""
for i in range(len(df1['body'])):
   df1.loc[[i],['expan_contra']] = contractions.fix(df1['body'][i])

# Step2: Expanding the Abbrevations by comparing the words with slang file

In [ ]:
def expabr(user_string):
    user_string = user_string.split(" ")
    # File path which consists of Abbreviations.
    fileName = "/content/drive/MyDrive/Colab Notebooks/Data/slang.txt"
    j = 0

    for _str in user_string:
      # File Access mode [Read Mode]
      accessMode = "r"
      with open(fileName, accessMode) as myCSVfile:
      # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
          dataFromFile = csv.reader(myCSVfile, delimiter="=")
          for row in dataFromFile:
# Check if selected word matches short forms[LHS] in text file.
            if _str.upper() == row[0]:  
              user_string[j] = row[1]   
          myCSVfile.close()
      j = j + 1
    # Replacing commas with spaces for final output.
    return ' '.join(user_string)

In [ ]:
df1['body_abbr']=""
for i in range(len(df1['body'])): 
  df1['body_abbr'][i] = expabr(df1['expan_contra'][i])

# Expanding the symbols

In [ ]:
def expsym(user_string):
    user_string = re.sub(";", " ; ", user_string)
    user_string = re.sub("\\?", " ? ", user_string)
    user_string = re.sub("=", " equals to ", user_string)
    user_string = re.sub(":", " : ", user_string)
    user_string = re.sub(">", " greater than ", user_string)
    user_string = re.sub("<", " lesser than ", user_string)
    user_string = re.sub("_", " _ ", user_string)
    user_string = re.sub("[^0-9a-zA-Z=\s:;?_-]+", " ",user_string)
    user_string = ' '.join(re.split('(\d+)', user_string))
    user_string = user_string.split(" ")
    # File path which consists of Abbreviations.
    fileName = "/content/drive/MyDrive/Colab Notebooks/Data/symbols.txt"
    j = 0

    for _str in user_string:
      # File Access mode [Read Mode]
      accessMode = "r"
      with open(fileName, accessMode) as myCSVfile:
      # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
          dataFromFile = csv.reader(myCSVfile, delimiter="=")
          for row in dataFromFile:
# Check if selected word matches short forms[LHS] in text file.
            if _str == row[0]:  
              user_string[j] = row[1]   
          myCSVfile.close()
      j = j + 1
    # Replacing commas with spaces for final output.
    return ' '.join(user_string)

In [ ]:
df1['body_syexp']=""
for i in range(len(df1['body'])):
  df1['body_syexp'][i] = expsym(df1['body_abbr'][i])


# Step4: Tokenising the sentence

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
def senttok(user_string):
  tokenized_text=sent_tokenize(user_string)
  return tokenized_text

In [ ]:
df1['sen_token'] = ""
for i in range(len(df1['body'])):
  df1['sen_token'][i] = senttok(df1['body_syexp'][i])

# Step3 : Tokenising the words

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def wordtok(user_string):
  tokenized_word=word_tokenize(user_string)
  return tokenized_word

In [ ]:
df1['word_token']=""
for i in range(len(df1['body'])):
 df1['word_token'][i] = wordtok(df1['body_syexp'][i])

# Creating single text corpus

In [ ]:
df2 = df1[['body_syexp']]
ndf = df2.unstack().to_frame().T

In [ ]:
ndf['Design conversation dataset'] = ndf[ndf.columns[0:]].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
ndf['Design conversation dataset'].to_csv('/content/drive/MyDrive/Colab Notebooks/Data/designcorpus_data', encoding='utf-8')

# Step4: Remove the stopwords

In [ ]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [ ]:
df1["no_stpwrd"] = ""
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in text if word not in stop_words])


df1["no_stpwrd"] = df1["word_token"].apply(lambda text: remove_stopwords(text))
df1["no_stpwrd"].head(25)

0     Hi piston My crankshaft depends bore diameter ...
1                                                 thank
2                     How low go piston bore diameter ?
3     factorofsafety values hundreds suspect depends...
4     Hi flywheel My crankshaft depends flywheel sha...
5                                                min 40
6                    What value works best simulation ?
7                              Do value bore diameter ?
8                               let play around numbers
9     My flywheel also depends crankshaft bearing of...
10    I broadcasted recent simulation definitely fin...
11                                          Sounds good
12                                        yes broadcast
13          Sounds good I tried middle values broadcast
14                                    Thank information
15    Hey I conrod I curious thought good piston dia...
16                                      45 seems decent
17    How flywheel thickness factor bearing supp

In [ ]:
df1["stpwrd"] = ""
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in text if word in stop_words])

#check this
df1["stpwrd"] = df1[ " "].apply(lambda text: remove_stopwords(text))
df1["stpwrd"].head(25)

# Convert to lowercase

In [ ]:
# Create a new column as not to interfere with the existing data
df1['lw_case_wordt']= ""

def lower_casing(text):
  return " ".join(text).lower()

df1['lw_case_wordt'] = df1["no_stpwrd"].apply(lambda text: lower_casing(text))
df1['lw_case_wordt'].head()

In [ ]:
# Create a new column as not to interfere with the existing data
df1['lw_case']= ""

def lower_casing(text):
  return text.lower()

df1['lw_case'] = df1['no_stpwrd'].apply(lambda text: lower_casing(text))
df1['lw_case'].head()

# Stemming the Words

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem_words(text):
  """function to stemm the words"""
  return ps.stem(text)

In [ ]:
df1['stemm'] =df1["lw_case"].apply(lambda text: stem_words(text))

In [ ]:
df1['stemm_stpword'] =df1["lw_case_wordt"].apply(lambda text: stem_words(text))

# Import packages for Lemmatize the words

In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

# Lemmatize body without stop words using Verb and Noun tags

In [ ]:
def lemmatize_words(text):
  """custom function to lemmatise words"""
  pos_tagged_text = nltk.pos_tag(wordtok(text))
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.VERB)) for word, pos in pos_tagged_text])

df1["lemma"] = df1["lw_case_wordt"].apply(lambda text: lemmatize_words(text))
df1["lemma"] .head()

In [ ]:
def lemmatize_words(text):
  """custom function to lemmatise words"""
  pos_tagged_text = nltk.pos_tag(wordtok(text))
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df1["lemma"] = df1["lemma"].apply(lambda text: lemmatize_words(text))
df1["lemma"] .head()

# Lemmatize body with stop words using Verb and Noun tags

In [ ]:
def lemmatize_words(text):
  """custom function to lemmatise words"""
  pos_tagged_text = nltk.pos_tag(wordtok(text))
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.VERB)) for word, pos in pos_tagged_text])

df1["lemma_stpword"] = df1["lw_case_wordt"].apply(lambda text: lemmatize_words(text))
df1["lemma_stpword"] .head()

In [ ]:
def lemmatize_words(text):
  """custom function to lemmatise words"""
  pos_tagged_text = nltk.pos_tag(wordtok(text))
  return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df1["lemma_stpword"] = df1["lemma_stpword"].apply(lambda text: lemmatize_words(text))
df1["lemma_stpword"] .head()

In [ ]:
intent_data = df1[["no_stpwrd"]]

In [ ]:
intent_data["intent"]=df2[["intent_2"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
intent_data.columns

Index(['no_stpwrd', 'intent'], dtype='object')

In [ ]:
intent_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Intentfile2.csv')